In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import types
from pyspark.context import SparkContext
import pandas as pd

In [2]:
credentials_location="credentials/dtc-de-380810-a07bfe4e7b8b.json"

In [3]:
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "libs/gcs-connector-hadoop3-2.2.12-shaded.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [4]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

23/04/06 21:46:04 WARN Utils: Your hostname, smarteez-HP-ProBook-440-G8-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp0s20f3)
23/04/06 21:46:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/06 21:46:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
green_schema=types.StructType([\
    types.StructField('VendorID', types.IntegerType(), True),\
    types.StructField('lpep_pickup_datetime', types.TimestampType(), True),\
    types.StructField('lpep_dropoff_datetime', types.TimestampType(), True),\
    types.StructField('store_and_fwd_flag', types.StringType(), True),\
    types.StructField('RatecodeID', types.IntegerType(), True),\
    types.StructField('PULocationID', types.IntegerType(), True),\
    types.StructField('DOLocationID', types.IntegerType(), True),\
    types.StructField('passenger_count', types.IntegerType(), True),\
    types.StructField('trip_distance', types.DoubleType(), True),\
    types.StructField('fare_amount', types.DoubleType(), True),\
    types.StructField('extra', types.DoubleType(), True),\
    types.StructField('mta_tax', types.DoubleType(), True),\
    types.StructField('tip_amount', types.DoubleType(), True),\
    types.StructField('tolls_amount', types.DoubleType(), True),\
    types.StructField('ehail_fee', types.DoubleType(), True),\
    types.StructField('improvement_surcharge', types.DoubleType(), True),\
    types.StructField('total_amount', types.DoubleType(), True),\
    types.StructField('payment_type', types.IntegerType(), True),\
    types.StructField('trip_type', types.IntegerType(), True),\
    types.StructField('congestion_surcharge', types.DoubleType(), True)])

In [7]:
df_green=spark.read.option('header','true').csv("gs://prefect-data-engineering-zoomcamp/data/green/2020/01/green_tripdata_2020_01.csv.gz")

In [9]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2019-12-18 15:52:30|  2019-12-18 15:54:39|                 N|         1|         264|         264|              5|          .00|        3.5|  0.5|    0.